# Configurations

A configuration is essentially a tree of (coordinate) frames, where each frame can have an associated shape, joint, inertia, etc. This tutorial introduces first steps to creating & loading configurations in rai, setting/getting the joint and frame state, computing features, and handling the view window.

Please see the [Script:Configurations](https://marctoussaint.github.io/robotics-course/script/script.html#scene-robot-description) of configurations and degrees-of-freedom.

## Building a configuration from scratch

Let's first build a configuration from scratch by adding frames and setting properties such as pose, shape and parentship of frames.

In [1]:
from robotic import ry
import numpy as np
import time
print('version:', ry.__version__, ry.compiled())

version: 0.0.22 compile time: Oct 10 2023 22:38:22


The following creates an empty configuration, and opens a view window for it.

Tip: Make the view window appear "Always On Top" (right click on the window bar).

In [2]:
C = ry.Config()
C.view()

0

A configuration is essentially a tree (or forrest) of frames. Let's add a frame and set positions manually. We declare it a **marker** frame, which means it has no real shape, but is displayed using its base vectors in the view.

In [3]:
C.clear()
f = C.addFrame(name='first')
f.setShape(type=ry.ST.marker, size=[.3])
f.setPosition([0.,0.,.5])
f.setQuaternion([1., .3, .0, .0]) # more about quaternions below
print('frame name:', f.name, 'pos:', f.getPosition(), 'quat:', f.getQuaternion())
C.view()

frame name: first pos: [0.  0.  0.5] quat: [0.95782629 0.28734789 0.         0.        ]


0

Let's add a second frame, but with first as parent and with a hinge joint!

In [4]:
f = C.addFrame(name='second', parent='first')
f.setJoint(ry.JT.hingeX)
f.setShape(type=ry.ST.marker, size=[.4])
f.setColor([1,0,0])
print('frame name:', f.name, 'pos:', f.getPosition(), 'quat:', f.getQuaternion())
C.view()

frame name: second pos: [0.  0.  0.5] quat: [-0.95782629 -0.28734789 -0.         -0.        ]


0

Since we now have a configuration with a joint, we can articulate it:

In [5]:
q = C.getJointState()
q[0] = q[0] + .2
C.setJointState(q)
print('joint state:', q)
C.view()

joint state: [0.2]


0

For illustration, let's add various other child frames with shapes to the 'second' frame and animate it with a trivial loop:

In [6]:
C.addFrame('ball', 'second') .setShape(ry.ST.sphere, [.1]) .setColor([1.,.5,.0]) .setRelativePosition([-.3,.0,.2])
C.addFrame('box', 'second') .setShape(ry.ST.ssBox, [.3,.2,.1,.02]) .setColor([.5,1.,.0]) .setRelativePosition([.0,.0,.2])
C.addFrame('capsule', 'second') .setShape(ry.ST.capsule, [.3, .05]) .setColor([.0,1.,.5]) .setRelativePosition([.3,.0,.2])
for t in range(100):
    C.setJointState([np.cos(.1*t)])
    C.view()
    time.sleep(.1)

The following lists all predefined shape types:

In [7]:
ry.ST.__members__.keys()

dict_keys(['none', 'box', 'sphere', 'capsule', 'mesh', 'cylinder', 'marker', 'pointCloud', 'ssCvx', 'ssBox', 'ssCylinder', 'ssBoxElip', 'quad', 'camera', 'sdf'])

For most of these, the `size` fully specifies them (box: 3 widths, sphere: radius, capsule: [length radius], cylinder: [length radius], marker: [axes lengths]). The `ssBox` is a sphere-swept-Box with a 4D size: [3 widths, corner radius], which is used a lot. Also mesh shapes can be set manually from python (`setMesh`) by directly giving vertex and triangle arrays. `ssCvx` means sphere-swept convex mesh, which is specified by a mesh and 1D size (radius), and also a very useful shape type.

Let's also briefly list the possible joint types:

In [8]:
ry.JT.__members__.keys()

dict_keys(['hingeX', 'hingeY', 'hingeZ', 'transX', 'transY', 'transZ', 'transXY', 'trans3', 'transXYPhi', 'transYPhi', 'universal', 'rigid', 'quatBall', 'phiTransXY', 'XBall', 'free', 'generic', 'tau'])

Please see the [Script:Kinematics](https://marctoussaint.github.io/robotics-course/script/script.html#kinematics) on the basics of such joints (=parameterized transformations). The `quatBall` perhaps deserves special explanation: it realizes a ball joint by introducing the 4D quaternion as joint state - this sometimes requires special attention. But the rai code implements correct Jacobians (accounting also for implicit normalization) w.r.t. the quaternion parameterization. The `free` joint is a 7D joint with 3D position and `quatBall`.

## Specifying transformations: Quaternions & turtle strings

See the [Script:Transformations](https://marctoussaint.github.io/robotics-course/script/script.html#d-transformations-rotations-quaternions) on background on 3D transformations, rotations, and quaternions. We use the (w,x,y,z) convention for quaternions. Whenever you set a quaternion manually, it is internally normalized. So setting a quaterniong (1,1,0,0) is totally fine and means 90-degrees around x:

In [9]:
C.clear()
C.addFrame('A') .setShape(ry.ST.marker, [.3]) .setPosition([0.,0.,.5]) .setQuaternion([1., 1., .0, .0])
C.view()

0

But to specify transformations as a human, it is easier to use what I call turtle commands: a string of 'translate', 'rotate' commands that is sequenced to define a transformation:

In [10]:
C.clear()
f = C.addFrame('A') .setShape(ry.ST.marker, [.3])
f.setPose('t(0 0 .5) d(30 1 0 0) t(0 0 .5) d(30 1 0 0)') #t=translate, #d=rotate-by-degrees around axis
C.view()

0

Also rpy-Euler angles `E`, rotation-by-radians `r`, and quaternions `q` can be used in turtle strings.

## Loading existing configurations

You will usually load predefined configurations from file, and then perhaps edit and combine them manually. The rai package has several robot models predefined and installed in the `raiPath`. They typically have the `.g`-file extension (see [Script:gFiles](https://marctoussaint.github.io/robotics-course/script/script.html#graph-yaml-files) on this file format, which is a yaml-conform graph file). Converting from URDF to configuration files is possible (see advanced tutorial).

In [11]:
C.clear()
C.addFile(ry.raiPath('panda/panda.g'))
C.view()

0

Let's add a second panda, but to avoid frame name conflicts we prefix all frame names. That's often important when importing multiple models into a single configuration. The can then also move the second robot simply by redefining the position of its base frame.

In [12]:
C.addFile(ry.raiPath('panda/panda.g'), 'r_')
base_r = C.getFrame('r_panda_base')
base_r.setPosition([.0, .5, .0])
C.view()

0

These models have several joints. We can get the joint state of the full configuration:

In [13]:
print(C.getJointState())
print('joints:', C.getJointNames())
print('frames:', C.getFrameNames())

[ 0.   -1.    0.   -2.    0.    2.    0.    0.05  0.   -1.    0.   -2.
  0.    2.    0.    0.05]
joints: ['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_finger_joint1panda_finger_joint2', 'r_panda_joint1', 'r_panda_joint2', 'r_panda_joint3', 'r_panda_joint4', 'r_panda_joint5', 'r_panda_joint6', 'r_panda_joint7', 'r_panda_finger_joint1r_panda_finger_joint2']
frames: ['panda_base', 'panda_link0', 'panda_link0_0', 'panda_joint1_origin', 'panda_joint1', 'panda_link1', 'panda_link1_0', 'panda_joint2_origin', 'panda_joint2', 'panda_link2', 'panda_link2_0', 'panda_joint3_origin', 'panda_joint3', 'panda_link3', 'panda_link3_0', 'panda_joint4_origin', 'panda_joint4', 'panda_link4', 'panda_link4_0', 'panda_joint5_origin', 'panda_joint5', 'panda_link5', 'panda_link5_0', 'panda_joint6_origin', 'panda_joint6', 'panda_link6', 'panda_link6_0', 'panda_joint7_origin', 'panda_joint7', 'panda_link7', 'panda_link7_0', 'panda_joint8_or

Let's animate - without respect for limits or collisions! (E.g., the fingers go out of limits.)

In [14]:
q0 = C.getJointState()
for t in range(20):
    q = q0 + .1*np.random.randn(q0.shape[0])
    C.setJointState(q)
    C.view()
    time.sleep(.2)

## Features: computing geometric properties
For every frame we can query its pose:

In [15]:
f = C.getFrame('r_gripper')
print('gripper pos:', f.getPosition())
print('gripper quat:', f.getQuaternion())
print('gripper rot:', f.getRotationMatrix())

gripper pos: [0.28285113 0.48303487 0.78372153]
gripper quat: [-0.82227892 -0.21950445  0.40232861  0.33735273]
gripper rot: [[ 0.44864964  0.37817024 -0.80975353]
 [-0.73142193  0.67602185 -0.08953445]
 [ 0.51355182  0.63244108  0.57989897]]


The above provides basic forward kinematics: After `setJointState` you can query the pose of any configuration frame.

However, there is a more general way to query *features* (see [Script:Features](https://marctoussaint.github.io/robotics-course/script/script.html#general-concept-of-differentiable-features)), i.e. properties of the configuration in a differentiable manner. You might not use this directly often; but defining differentiable features is the foundation of how optimization problems are formulated, which you will need a lot.

Here are some example features to evaluate:

In [16]:
[y,J] = C.eval(ry.FS.position, ['gripper'])
print('position of gripper:', y, '\nJacobian:', J)

position of gripper: [0.27017462 0.03622352 0.84999348] 
Jacobian: [[-3.62235225e-02  5.13463103e-01 -7.02622624e-02 -2.71168832e-01
  -1.27611357e-02  2.56483270e-02  2.60208521e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.70174616e-01  6.03150204e-02  5.76809785e-01 -2.28430634e-02
   1.64795742e-01  2.69416090e-02 -6.93889390e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.72555698e-01 -3.69637629e-03  4.88346517e-01
  -1.75743534e-02  2.34009216e-01  1.73472348e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [17]:
# negative(!) distance between two convex shapes (or origin of marker)
C.eval(ry.FS.negDistance, ['panda_coll7', 'r_panda_coll7'])

(array([-0.287672]),
 array([[ 8.92832227e-02,  5.19279493e-02,  5.58803809e-01,
         -3.76354350e-02, -2.31257978e-02,  1.11019864e-03,
         -3.11061377e-17, -0.00000000e+00, -7.07358385e-02,
          3.18319343e-02, -5.48892145e-01, -7.56432874e-03,
          6.87288685e-02,  6.43205889e-04,  1.36335428e-17,
         -0.00000000e+00]]))

In [18]:
# the x-axis of the given frame in world coordinates
C.eval(ry.FS.vectorX, ['gripper'])

(array([ 0.35672715, -0.68401201,  0.63629656]),
 array([[ 0.68401201,  0.6319515 ,  0.32046993, -0.61004922,  0.49726775,
         -0.55852166, -0.31342281,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.35672715,  0.07423351,  0.72343628, -0.07620321, -0.28012681,
         -0.01113349, -0.72925977,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        , -0.27449095,  0.59802111,  0.26009446, -0.57991671,
          0.3011558 , -0.60823213,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]]))

## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [19]:
C.setJointState(q0)
C.view()

0

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [20]:
X0 = C.getFrameState()
print('frame state: ', X0)

frame state:  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01  7.07106781e-01
  -7.07106781e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33000000e-01 

Let's do a very questionable thing: adding .1 to all numbers in the frame matrix!

In [21]:
X = X0 + .1
C.setFrameState(X)
C.view()

0

That totally broke the original design of the robot! Setting global frame states overwrites the relative transformations between frames.

(Also, the rows of X have non-normalized quaternions! These are normalized when setting the frame state.)

Let's reset:

In [22]:
C.setFrameState(X0)
C.view()

0

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-select joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q. The frame state is not affected by such a selection of active joints.

In [23]:
C.selectJoints(['panda_joint1', 'panda_joint1'])
print('joint state: ', C.getJointState())
print('joint names: ', C.getJointNames() )

joint state:  [0.]
joint names:  ['panda_joint1']


In [24]:
C.selectJoints([], True)
print('joint state: ', C.getJointState())
print('joint names: ', C.getJointNames() )

joint state:  [ 0.      -1.00001  0.      -2.00001  0.       2.00001  0.       0.05
  0.      -1.00001  0.      -2.00001  0.       2.00001  0.       0.05   ]
joint names:  ['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_finger_joint1panda_finger_joint2', 'r_panda_joint1', 'r_panda_joint2', 'r_panda_joint3', 'r_panda_joint4', 'r_panda_joint5', 'r_panda_joint6', 'r_panda_joint7', 'r_panda_finger_joint1r_panda_finger_joint2']


## View interaction and releasing objects

You can close and re-open the view window

In [25]:
C.view_close()

In [26]:
# things are still there
C.view(pause=False, message='this is a message')

0

For user interaction it is often useful to wait for a keypress (making `view` a blocking call):

In [27]:
keypressed = C.view(True, 'press some key!')
print('pressed key:', keypressed, chr(keypressed))

pressed key: 121 y


You can also get a screenshot:

In [28]:
img = C.view_getScreenshot()
print(type(img), img.shape)

<class 'numpy.ndarray'> (400, 400, 3)


In [29]:
import matplotlib.pyplot as plt
plt.imshow(img)
plt.show()

And release everything, including closing the view

In [30]:
del C